<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mBackground

A large percentage of the Montage modules focus on processing a set of images (often retrieved from a mission archive); projecting them to a common frame, adjusting their background levels as a set, and coadding them into a mosaic.

This page is focused on the details of one of these modules: mBackground.  For a broader context, please see <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands.

<b>Note:</b> The MontagePy python package has no external dependencies. We include packages like AstroPy on this page to aid in visualizing MontagePy package results.


In [1]:
from MontagePy.main import mBackground, mViewer

help(mBackground)

Help on built-in function mBackground in module MontagePy.main:

mBackground(...)
    mBackground subtracts a planar background from a a FITS image.
    
    Parameters
    ----------
    input_file : str
        Input FITS file.
    output_file : str
        Output background-removed FITS file.
    A : float
        A coefficient in (A*x + B*y + C) background level equation.
    B : float
        B coefficient in (A*x + B*y + C) background level equation.
    C : float
        C level in (A*x + B*y + C) background level equation.
    noAreas : bool, optional
        Don't process associated area images.
    debug : int, optional
        Debugging output level.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).
    time : float
        Total processing time (sec).



## mBackground Example

mBackground simply removes a planar background from an image.  It can be used directly but the more normal case is where it is called in conjunction with another utility like mFitplane or mBgModel to remove a background fitted to the image itself or determined as part of global background rectification.

The arguments to mBackground are the input image, output image name, and three numeric coefficients defining the plane to subtract:  Ax + By + C (where x and y are the pixel coordinates of in the image):

In [2]:
rtn = mBackground('M17/corrected/hdu0_2mass-atlas-990502s-j1340186.fits', 
                  'work/M17/hdu0_2mass-atlas-990502s-j1340186_background.fits', 
                  0.01, 0.02, 3.)
print(rtn)

{'status': '0', 'time': 0.0}


## Before and After

In this case, we are subtracting a plane from an image that already has a good background, so this is just for illustration.  Here is the original image.

With this background subtracted off, it gets harder to find a stretch where the image looks good.  Note that the image background is darkest in the lower left and brightest in the upper right, as you would expect with this background (and the display being a negative).

In [3]:
from IPython.display import display, HTML

rtn = mViewer('-ct 1 -gray input/corrected/hdu0_2mass-atlas-990502s-j1340186.fits \
               -2s max gaussian-log -out work/M17/hdu0_2mass-atlas-990502s-j1340186.png',
              '', mode=2)

rtn = mViewer('-ct 1 -gray work/M17/hdu0_2mass-atlas-990502s-j1340186_background.fits \
               -2s max gaussian-log -out work/M17/hdu0_2mass-atlas-990502s-j1340186_background.png',
              '', mode=2)

display(HTML('<table><tr> \
              <td><img src="work/M17/hdu0_2mass-atlas-990502s-j1340186.png"></td> \
              <td><img src="work/M17/hdu0_2mass-atlas-990502s-j1340186_background.png"></td></tr></table>'))


,


The added background slope on the right may not look like much in its own right but it has a strong effect on our ability to find a good stretch for the image display; much of the low-level structure in the image on the left is small compared to the amount of difference we have added with the slope in the background.

&nbsp;<p/>

## mBackground Error Handling

If mBackground encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies an image that doesn't exist: 

In [4]:
rtn = mBackground('M17/corrected/unknown.fits', 
                  'work/M17/hdu0_2mass-atlas-990502s-j1340186_background.fits', 
                  0.01, 0.02, 3.)
print(rtn)

{'status': '1', 'msg': b'File M17/corrected/unknown.fits not found.'}


&nbsp;


## Classic Montage: mBackground as a Stand-Alone Program

### mBackground Unix/Windows Command-line Arguments

<p>mBackground can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mBackground [-d level] [-n(o-areas)] [-s statusfile] in.fits out.fits A B C | mBackground [-t(able-mode)] [-d level] [-n(o-areas)] [-s statusfile] in.fits out.fits images.tbl corrfile.tbl
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mBackground can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mBackground                                                          */
/*                                                                       */
/*  Montage is a set of general reprojection / coordinate-transform /    */
/*  mosaicking programs.  Any number of input images can be merged into  */
/*  an output FITS file.  The attributes of the input are read from the  */
/*  input files; the attributes of the output are read a combination of  */
/*  the command line and a FITS header template file.                    */
/*                                                                       */
/*  This module, mBackground, removes a background plane from a single   */
/*  projected image.                                                     */
/*                                                                       */
/*   char  *input_file     Input FITS file                               */
/*   char  *output_file    Output background-removed FITS file           */
/*                                                                       */
/*   double A              A coefficient in (A*x + B*y + C) background   */
/*                         level equation                                */
/*   double B              B coefficient in (A*x + B*y + C) background   */
/*                         level equation                                */
/*   double C              C level in (A*x + B*y + C) background         */
/*                         level equation                                */
/*                                                                       */
/*   int    noAreas        Don't process associated area images          */
/*                                                                       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mBackgroundReturn *mBackground(char *input_file, char *ofile, double A, double B, double C, int noAreasin, int debug)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mBackgroundReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   double time;          // Run time (sec)   
};
</pre>